In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

In [43]:
mnist_dataset, mnist_info = tfds.load(name="mnist", with_info=True, as_supervised=True)

W0725 22:33:05.076838 140061408048960 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


In [44]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    version=1.0.0,
    description='The MNIST database of handwritten digits.',
    urls=['https://storage.googleapis.com/cvdf-datasets/mnist/'],
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

In [81]:
# SPlit dataset into train and test
x_train, x_test = mnist_dataset["train"], mnist_dataset["test"]
# define size of cross validation dataset
n_cross_validation = 0.1*mnist_info.splits["train"].num_examples
n_cross_validation = tf.cast(n_cross_validation, tf.int64)

# define size of test dataset
n_test = mnist_info.splits["test"].num_examples
n_test = tf.cast(n_test, tf.int64)

# define a function to standardize the values
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

# Apply dataset.map() function to transform the data
scaled_train_valid_data = x_train.map(scale)
scaled_test_data = x_test.map(scale)

# Shuffle the training dataset
buffer_size = 10000

# Shuffle train and validation datasest
shuffled_train_validation_data = scaled_train_valid_data.shuffle(buffer_size)
# Select first n_cross_validation samples as cross_validation dataset
validation_data = shuffled_train_validation_data.take(n_cross_validation)
# Slip first n_cross_validation samples to form the train_data
train_data = shuffled_train_validation_data.skip(n_cross_validation)

### Set up batches for sgd

In [82]:
batch_size = 1000
# define training batches of batch size 100
train_data = train_data.batch(batch_size)
# define a single batch of all validation data
validation_data = validation_data.batch(n_cross_validation)
# define a single batch of all test data
test_data = scaled_test_data.batch(n_test)

### Split into input data and targets

In [83]:
x_train, y_train = next(iter(train_data))
x_valid, y_valid = next(iter(validation_data))
x_test, y_test = next(iter(test_data))

### Build model

In [84]:
input_size = 28*28
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

### Compile the model

In [85]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])


### Train the model

In [ ]:
%%time
NUM_EPOCH = 10
model.fit(train_data, epochs=NUM_EPOCH, validation_data=(x_valid, y_valid), verbose=2)

Epoch 1/10
54/54 - 10s - loss: 0.0750 - accuracy: 0.9775 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
54/54 - 6s - loss: 0.0622 - accuracy: 0.9815 - val_loss: 0.0620 - val_accuracy: 0.9808
Epoch 3/10
54/54 - 4s - loss: 0.0516 - accuracy: 0.9851 - val_loss: 0.0527 - val_accuracy: 0.9853
Epoch 4/10
54/54 - 5s - loss: 0.0439 - accuracy: 0.9874 - val_loss: 0.0470 - val_accuracy: 0.9868
Epoch 5/10
54/54 - 5s - loss: 0.0369 - accuracy: 0.9894 - val_loss: 0.0412 - val_accuracy: 0.9900
Epoch 6/10
54/54 - 5s - loss: 0.0318 - accuracy: 0.9907 - val_loss: 0.0387 - val_accuracy: 0.9900
Epoch 7/10
54/54 - 8s - loss: 0.0253 - accuracy: 0.9934 - val_loss: 0.0320 - val_accuracy: 0.9913
Epoch 8/10
54/54 - 4s - loss: 0.0216 - accuracy: 0.9939 - val_loss: 0.0305 - val_accuracy: 0.9922
Epoch 9/10


In [80]:
model.evaluate(test_data)

      1/Unknown - 1s 786ms/step - loss: 0.1138 - accuracy: 0.9753

[0.11383108049631119, 0.9753]